In [1]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np

from collections import defaultdict
from tqdm import tqdm 

tqdm.pandas()

In [2]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [3]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [4]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 469568/469568 [00:20<00:00, 23179.63it/s]


In [5]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [6]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]

In [7]:
career_paths

candidate_id
84556      [208, 348, 208, 344, 344, 344, 344, 344, 344, ...
84612                    [344, 335, 332, 344, 335, 222, 344]
84731                    [344, 208, 208, 345, 229, 345, 323]
85437      [344, 323, 344, 185, 177, 185, 185, 185, 189, ...
85888                                        [229, 229, 208]
                                 ...                        
8909379                                           [130, 177]
8909451                                           [222, 345]
8909472                                   [46, 46, 345, 168]
8909570                                           [351, 345]
8909578                                      [222, 185, 117]
Name: isco_code4, Length: 113724, dtype: object

In [8]:
switches = defaultdict(lambda: defaultdict(int))

for c, jobs in career_paths.iteritems():
    for i in range(len(jobs)):
        if i < len(jobs) - 1 and jobs[i] != jobs[i + 1]:
            switches[jobs[i]][jobs[i  + 1]] += 1

In [9]:
most_common = pd.Series({k: max(v, key=v.get) for k, v in switches.items()})

In [10]:
most_common.head()

208    348
348    208
344    345
345    344
185    177
dtype: int64

In [11]:
most_common_switch = career_paths.progress_apply(lambda x: x[-1] == most_common.loc[x[-2]]).mean()

100%|██████████| 113724/113724 [00:04<00:00, 25561.24it/s]


In [12]:
print(f"Most common switch accuracy: {most_common_switch * 100:>.2f}%")

Most common switch accuracy: 23.15%
